:warning:**IMPORTANT NOTICE**:warning:\
*Since the initial parameterisation and gradient descent optimisation are stochastic processes, the training of a neural network is not fully reproducible.*

*Therefore, it is not recommended to re-run this script as it will overwrite the original calibration of the neural network used in the work presented here.
The purpose of this script is solely to document the training procedure and can be copied as a template to fit other new neural networks.*

*To experiment with the models calibrated here, they can be loaded from the `saved_models` directory.*

# Systematic assesment of different hyperparameters to optimise model performance

Various hyperparameters are varied to asses their impact on the model performance in a semi-quantitative way.\
The hyperparameters that are varied are:
- Number of hidden layers / neurons --> model capacity
- Activation function
- Effect of regularization

The model is trained on the same dataset for each hyperparameter combination.

### Model capacity

These architectures are tested:
- **small**: 2 hidden layers with 16 neurons each
- **medium**: 2 hidden layers with 64 neurons each
- **medium deep**: 4 hidden layers with 64 neurons each
- **large**: 4 hidden layers with 128 neurons each
- **large deep**: 6 hidden layers with 128 neurons each

### Activation function

These activation functions are tested:
- relu
- elu
- leaky relu

### Regularisation

These regularisation methods are tested:
- L2 regularisation
- Dropout
- Batch normalisation
- Layer normalisation
- Combined L2 regularisation and dropout


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import h5py

from pathlib import Path
from keras.models import Sequential
from keras.layers import Dense, Normalization, BatchNormalization, LayerNormalization, Dropout
from keras.losses import MeanSquaredError
from keras.optimizers import Adam, schedules
from keras.metrics import MeanAbsoluteError, RootMeanSquaredError
from keras.callbacks import CSVLogger, EarlyStopping
from sklearn.model_selection import train_test_split

from ml_tb.normalisation import MinMaxScaler
from ml_tb.metrics import RMSE_denormalised_P, RMSE_denormalised_T

## Import data, pre-procesing and train/test split

For biotite composition only extract elements which are modelled to be present in biotite by ds55/62.\
Element-idx from hdf5 file:
```python
(0, 'O') (1, 'SI') (2, 'TI') (3, 'AL') (4, 'FE') (5, 'MG') (6, 'MN') (7, 'CA') (8, 'NA') (9, 'K') (10, 'H')
```
Only extract: Si, Ti, Al, Fe, Mg, Mn.\
K would be in modelled biotite, but is fixed at 1.0.

Validation set is **10%** of the training set. Approx. 6000 samples.

In [2]:
# load hdf file
file = Path("10022024_Biotite_ds62.hdf5")
with h5py.File(file, "r") as hdf_file:
    PT = np.array(hdf_file["PT"])
    full_biotite_composition = np.array(hdf_file["mineral_composition"])
    element_idx = hdf_file["mineral_composition"].attrs["element idx [apfu]"]

print(*enumerate(element_idx))

biotite_composition = full_biotite_composition[:,[1, 2, 3, 4, 6, 5]]
pt = PT

biotite_composition_train, biotite_composition_val, pt_train, pt_val = train_test_split(biotite_composition, pt, test_size=0.10, shuffle=True)

# normalise biotite composition & min-max scale PT
normalisation_biotite_composition = Normalization(axis=-1)
normalisation_biotite_composition.adapt(biotite_composition_train)

print(normalisation_biotite_composition.mean.numpy())
print(np.sqrt(normalisation_biotite_composition.variance.numpy()))

scaling_pt = MinMaxScaler(min=[1500, 400], max=[10000, 900], axis=0)
inv_scaling_pt = MinMaxScaler(min=[1500, 400], max=[10000, 900], axis=0, invert=True)
pt_train_norm = scaling_pt(pt_train)
pt_val_norm = scaling_pt(pt_val)

(0, 'O') (1, 'SI') (2, 'TI') (3, 'AL') (4, 'FE') (5, 'MG') (6, 'MN') (7, 'CA') (8, 'NA') (9, 'K') (10, 'H')

[[2.677286   0.11005508 1.524142   1.5183328  0.01044957 1.1597326 ]]
[[0.09299828 0.04669439 0.16551827 0.31402156 0.0094105  0.27365792]]


## Set-up global training parameters

Define a function to calculate RMSE for pressure and temperature for unscaled values to have an interpretable metric.

All models are trained for a maximum of 5000 epochs.\
Early stopping is used with a patience (no improvement of val loss) of 50 epochs.\
Inverse time learning rate decay is used for all models.

In [3]:
def RMSE_P(y_true, y_pred):
    return RMSE_denormalised_P(y_true, y_pred, inv_scaling_pt)


def RMSE_T(y_true, y_pred):
    return RMSE_denormalised_T(y_true, y_pred, inv_scaling_pt)

In [4]:
BATCH_SIZE = 500
STEPS_PER_EPOCH = len(biotite_composition_train) // BATCH_SIZE
MAX_EPOCHS = 5000

lr_schedule = schedules.InverseTimeDecay(0.001, decay_steps=STEPS_PER_EPOCH*1000, decay_rate=1, staircase=False)

LOSS = MeanSquaredError()
METRICS = [MeanAbsoluteError(), RootMeanSquaredError(), RMSE_P, RMSE_T]

## **Test 01:** Model capacity

In [5]:
CALLBACKS_SMALL = [CSVLogger("baseline_small.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_MEDIUM = [CSVLogger("baseline_medium.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_MEDIUMDEEP = [CSVLogger("baseline_mediumdeep.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_LARGE = [CSVLogger("baseline_large.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_LARGEDEEP = [CSVLogger("baseline_largedeep.log", append=False), EarlyStopping(monitor="loss", patience=50)]

In [6]:
OPT = Adam(lr_schedule)

small_model = Sequential()
small_model.add(normalisation_biotite_composition)
small_model.add(Dense(16, activation="elu"))
small_model.add(Dense(16, activation="elu"))
small_model.add(Dense(2))

small_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
small_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 6)                 13        
 on)                                                             
                                                                 
 dense (Dense)               (None, 16)                112       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 2)                 34        
                                                                 
Total params: 431 (1.69 KB)
Trainable params: 418 (1.63 KB)
Non-trainable params: 13 (56.00 Byte)
_________________________________________________________________


In [7]:
history = small_model.fit(biotite_composition_train, pt_train_norm,
                          batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                          validation_data=[biotite_composition_val, pt_val_norm],
                          callbacks=CALLBACKS_SMALL, verbose=False)
small_model.save(Path("saved_models", "small_model"))



INFO:tensorflow:Assets written to: saved_models\small_model\assets


INFO:tensorflow:Assets written to: saved_models\small_model\assets


In [8]:
OPT = Adam(lr_schedule)

medium_model = Sequential()
medium_model.add(normalisation_biotite_composition)
medium_model.add(Dense(64, activation="elu"))
medium_model.add(Dense(64, activation="elu"))
medium_model.add(Dense(2))

medium_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
medium_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 6)                 13        
 on)                                                             
                                                                 
 dense_3 (Dense)             (None, 64)                448       
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dense_5 (Dense)             (None, 2)                 130       
                                                                 
Total params: 4751 (18.56 KB)
Trainable params: 4738 (18.51 KB)
Non-trainable params: 13 (56.00 Byte)
_________________________________________________________________


In [9]:
history = medium_model.fit(biotite_composition_train, pt_train_norm,
                           batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                           validation_data=[biotite_composition_val, pt_val_norm],
                           callbacks=CALLBACKS_MEDIUM, verbose=False)
medium_model.save(Path("saved_models", "medium_model"))

INFO:tensorflow:Assets written to: saved_models\medium_model\assets


INFO:tensorflow:Assets written to: saved_models\medium_model\assets


In [10]:
OPT = Adam(lr_schedule)

mediumdeep_model = Sequential()
mediumdeep_model.add(normalisation_biotite_composition)
mediumdeep_model.add(Dense(64, activation="elu"))
mediumdeep_model.add(Dense(64, activation="elu"))
mediumdeep_model.add(Dense(64, activation="elu"))
mediumdeep_model.add(Dense(64, activation="elu"))
mediumdeep_model.add(Dense(2))

mediumdeep_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
mediumdeep_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 6)                 13        
 on)                                                             
                                                                 
 dense_6 (Dense)             (None, 64)                448       
                                                                 
 dense_7 (Dense)             (None, 64)                4160      
                                                                 
 dense_8 (Dense)             (None, 64)                4160      
                                                                 
 dense_9 (Dense)             (None, 64)                4160      
                                                                 
 dense_10 (Dense)            (None, 2)                 130       
                                                      

In [11]:
history = mediumdeep_model.fit(biotite_composition_train, pt_train_norm,
                               batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                               validation_data=[biotite_composition_val, pt_val_norm],
                               callbacks=CALLBACKS_MEDIUMDEEP, verbose=False)
mediumdeep_model.save(Path("saved_models", "mediumdeep_model"))

INFO:tensorflow:Assets written to: saved_models\mediumdeep_model\assets


INFO:tensorflow:Assets written to: saved_models\mediumdeep_model\assets


In [12]:
OPT = Adam(lr_schedule)

large_model = Sequential()
large_model.add(normalisation_biotite_composition)
large_model.add(Dense(128, activation="elu"))
large_model.add(Dense(128, activation="elu"))
large_model.add(Dense(128, activation="elu"))
large_model.add(Dense(128, activation="elu"))
large_model.add(Dense(2))

large_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
large_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 6)                 13        
 on)                                                             
                                                                 
 dense_11 (Dense)            (None, 128)               896       
                                                                 
 dense_12 (Dense)            (None, 128)               16512     
                                                                 
 dense_13 (Dense)            (None, 128)               16512     
                                                                 
 dense_14 (Dense)            (None, 128)               16512     
                                                                 
 dense_15 (Dense)            (None, 2)                 258       
                                                      

In [13]:
history = large_model.fit(biotite_composition_train, pt_train_norm,
                          batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                          validation_data=[biotite_composition_val, pt_val_norm],
                          callbacks=CALLBACKS_LARGE, verbose=False)
large_model.save(Path("saved_models", "large_model"))

INFO:tensorflow:Assets written to: saved_models\large_model\assets


INFO:tensorflow:Assets written to: saved_models\large_model\assets


In [14]:
OPT = Adam(lr_schedule)

largedeep_model = Sequential()
largedeep_model.add(normalisation_biotite_composition)
largedeep_model.add(Dense(128, activation="elu"))
largedeep_model.add(Dense(128, activation="elu"))
largedeep_model.add(Dense(128, activation="elu"))
largedeep_model.add(Dense(128, activation="elu"))
largedeep_model.add(Dense(128, activation="elu"))
largedeep_model.add(Dense(128, activation="elu"))
largedeep_model.add(Dense(2))

largedeep_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
largedeep_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 6)                 13        
 on)                                                             
                                                                 
 dense_16 (Dense)            (None, 128)               896       
                                                                 
 dense_17 (Dense)            (None, 128)               16512     
                                                                 
 dense_18 (Dense)            (None, 128)               16512     
                                                                 
 dense_19 (Dense)            (None, 128)               16512     
                                                                 
 dense_20 (Dense)            (None, 128)               16512     
                                                      

In [15]:
history = largedeep_model.fit(biotite_composition_train, pt_train_norm,
                              batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                              validation_data=[biotite_composition_val, pt_val_norm],
                              callbacks=CALLBACKS_LARGEDEEP, verbose=False)
largedeep_model.save(Path("saved_models", "largedeep_model"))

INFO:tensorflow:Assets written to: saved_models\largedeep_model\assets


INFO:tensorflow:Assets written to: saved_models\largedeep_model\assets


## **Test 02:** Activation function

All tests on the architecture determined to be optimal in test 01. --> medium_deep. 4 hidden layers with 64 neurons each.

In [16]:
CALLBACKS_elu = [CSVLogger("baseline_elu.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_relu = [CSVLogger("baseline_relu.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_leaky_relu = [CSVLogger("baseline_leaky_relu.log", append=False), EarlyStopping(monitor="loss", patience=50)]

In [17]:
OPT = Adam(lr_schedule)

elu_model = Sequential()
elu_model.add(normalisation_biotite_composition)
elu_model.add(Dense(64, activation="elu"))
elu_model.add(Dense(64, activation="elu"))
elu_model.add(Dense(64, activation="elu"))
elu_model.add(Dense(64, activation="elu"))
elu_model.add(Dense(2))

elu_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
elu_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 6)                 13        
 on)                                                             
                                                                 
 dense_23 (Dense)            (None, 64)                448       
                                                                 
 dense_24 (Dense)            (None, 64)                4160      
                                                                 
 dense_25 (Dense)            (None, 64)                4160      
                                                                 
 dense_26 (Dense)            (None, 64)                4160      
                                                                 
 dense_27 (Dense)            (None, 2)                 130       
                                                      

In [18]:
history = elu_model.fit(biotite_composition_train, pt_train_norm,
                        batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                        validation_data=[biotite_composition_val, pt_val_norm],
                        callbacks=CALLBACKS_elu, verbose=False)
elu_model.save(Path("saved_models", "elu_model"))

INFO:tensorflow:Assets written to: saved_models\elu_model\assets


INFO:tensorflow:Assets written to: saved_models\elu_model\assets


In [19]:
OPT = Adam(lr_schedule)

relu_model = Sequential()
relu_model.add(normalisation_biotite_composition)
relu_model.add(Dense(64, activation="relu"))
relu_model.add(Dense(64, activation="relu"))
relu_model.add(Dense(64, activation="relu"))
relu_model.add(Dense(64, activation="relu"))
relu_model.add(Dense(2))

relu_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
relu_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 6)                 13        
 on)                                                             
                                                                 
 dense_28 (Dense)            (None, 64)                448       
                                                                 
 dense_29 (Dense)            (None, 64)                4160      
                                                                 
 dense_30 (Dense)            (None, 64)                4160      
                                                                 
 dense_31 (Dense)            (None, 64)                4160      


                                                                 
 dense_32 (Dense)            (None, 2)                 130       
                                                                 
Total params: 13071 (51.06 KB)
Trainable params: 13058 (51.01 KB)
Non-trainable params: 13 (56.00 Byte)
_________________________________________________________________


In [20]:
history = relu_model.fit(biotite_composition_train, pt_train_norm,
                         batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                         validation_data=[biotite_composition_val, pt_val_norm],
                         callbacks=CALLBACKS_relu, verbose=False)
relu_model.save(Path("saved_models", "relu_model"))

INFO:tensorflow:Assets written to: saved_models\relu_model\assets


INFO:tensorflow:Assets written to: saved_models\relu_model\assets


In [21]:
OPT = Adam(lr_schedule)

leaky_relu_model = Sequential()
leaky_relu_model.add(normalisation_biotite_composition)
leaky_relu_model.add(Dense(64, activation="leaky_relu"))
leaky_relu_model.add(Dense(64, activation="leaky_relu"))
leaky_relu_model.add(Dense(64, activation="leaky_relu"))
leaky_relu_model.add(Dense(64, activation="leaky_relu"))
leaky_relu_model.add(Dense(2))

leaky_relu_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
leaky_relu_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 6)                 13        
 on)                                                             
                                                                 
 dense_33 (Dense)            (None, 64)                448       
                                                                 
 dense_34 (Dense)            (None, 64)                4160      
                                                                 
 dense_35 (Dense)            (None, 64)                4160      
                                                                 
 dense_36 (Dense)            (None, 64)                4160      
                                                                 
 dense_37 (Dense)            (None, 2)                 130       
                                                      

In [22]:
history = leaky_relu_model.fit(biotite_composition_train, pt_train_norm,
                               batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                               validation_data=[biotite_composition_val, pt_val_norm],
                               callbacks=CALLBACKS_leaky_relu, verbose=False)
leaky_relu_model.save(Path("saved_models", "leakyrelu_model"))

INFO:tensorflow:Assets written to: saved_models\leakyrelu_model\assets


INFO:tensorflow:Assets written to: saved_models\leakyrelu_model\assets


## **Test 03:** Regularisation

Test if regularisation can improve performance if applied to models with the best training set performance determined in tests 01 and 02.\
Model architecture: large_deep. 6 hidden layers with 128 neurons each.\
Activation function: relu.


The idea behind this test is to see if regularisation can prevent overfitting and improve the performance on the validation set beyond what is achieved by smaller models.


In [23]:
CALLBACKS_noReg = [CSVLogger("baseline_noReg.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_L2 = [CSVLogger("baseline_L2.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_dropout = [CSVLogger("baseline_dropout.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_BatchNorm = [CSVLogger("baseline_BatchNorm.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_LayerNorm = [CSVLogger("baseline_LayerNorm.log", append=False), EarlyStopping(monitor="loss", patience=50)]
CALLBACKS_L2_dropout = [CSVLogger("baseline_L2_dropout.log", append=False), EarlyStopping(monitor="loss", patience=50)]

In [24]:
OPT = Adam(lr_schedule)

noReg_model = Sequential()
noReg_model.add(normalisation_biotite_composition)
noReg_model.add(Dense(128, activation="relu"))
noReg_model.add(Dense(128, activation="relu"))
noReg_model.add(Dense(128, activation="relu"))
noReg_model.add(Dense(128, activation="relu"))
noReg_model.add(Dense(128, activation="relu"))
noReg_model.add(Dense(128, activation="relu"))
noReg_model.add(Dense(2))

noReg_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
noReg_model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 6)                 13        
 on)                                                             
                                                                 
 dense_38 (Dense)            (None, 128)               896       
                                                                 
 dense_39 (Dense)            (None, 128)               16512     
                                                                 
 dense_40 (Dense)            (None, 128)               16512     
                                                                 
 dense_41 (Dense)            (None, 128)               16512     
                                                                 
 dense_42 (Dense)            (None, 128)               16512     
                                                      

In [25]:
history = noReg_model.fit(biotite_composition_train, pt_train_norm,
                          batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                          validation_data=[biotite_composition_val, pt_val_norm],
                          callbacks=CALLBACKS_noReg, verbose=False)
noReg_model.save(Path("saved_models", "noReg_model"))

INFO:tensorflow:Assets written to: saved_models\noReg_model\assets


INFO:tensorflow:Assets written to: saved_models\noReg_model\assets


In [26]:
OPT = Adam(lr_schedule)

L2_model = Sequential()
L2_model.add(normalisation_biotite_composition)
L2_model.add(Dense(128, activation="relu", kernel_regularizer="l2"))
L2_model.add(Dense(128, activation="relu", kernel_regularizer="l2"))
L2_model.add(Dense(128, activation="relu", kernel_regularizer="l2"))
L2_model.add(Dense(128, activation="relu", kernel_regularizer="l2"))
L2_model.add(Dense(128, activation="relu", kernel_regularizer="l2"))
L2_model.add(Dense(128, activation="relu", kernel_regularizer="l2"))
L2_model.add(Dense(2))

L2_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
L2_model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 6)                 13        
 on)                                                             
                                                                 
 dense_45 (Dense)            (None, 128)               896       
                                                                 
 dense_46 (Dense)            (None, 128)               16512     
                                                                 
 dense_47 (Dense)            (None, 128)               16512     
                                                                 
 dense_48 (Dense)            (None, 128)               16512     
                                                                 
 dense_49 (Dense)            (None, 128)               16512     
                                                      

In [27]:
history = L2_model.fit(biotite_composition_train, pt_train_norm,
                         batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                         validation_data=[biotite_composition_val, pt_val_norm],
                         callbacks=CALLBACKS_L2, verbose=False)
L2_model.save(Path("saved_models", "L2_model"))

INFO:tensorflow:Assets written to: saved_models\L2_model\assets


INFO:tensorflow:Assets written to: saved_models\L2_model\assets


In [28]:
OPT = Adam(lr_schedule)

dropout_model = Sequential()
dropout_model.add(normalisation_biotite_composition)
dropout_model.add(Dense(128, activation="relu"))
dropout_model.add(Dropout(0.2))
dropout_model.add(Dense(128, activation="relu"))
dropout_model.add(Dropout(0.2))
dropout_model.add(Dense(128, activation="relu"))
dropout_model.add(Dropout(0.2))
dropout_model.add(Dense(128, activation="relu"))
dropout_model.add(Dropout(0.2))
dropout_model.add(Dense(128, activation="relu"))
dropout_model.add(Dropout(0.2))
dropout_model.add(Dense(128, activation="relu"))
dropout_model.add(Dropout(0.2))
dropout_model.add(Dense(2))

dropout_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
dropout_model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 6)                 13        
 on)                                                             
                                                                 
 dense_52 (Dense)            (None, 128)               896       
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_53 (Dense)            (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_54 (Dense)            (None, 128)               16512     
                                                     

In [29]:
history = dropout_model.fit(biotite_composition_train, pt_train_norm,
                         batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                         validation_data=[biotite_composition_val, pt_val_norm],
                         callbacks=CALLBACKS_dropout, verbose=False)
dropout_model.save(Path("saved_models", "dropout_model"))

INFO:tensorflow:Assets written to: saved_models\dropout_model\assets


INFO:tensorflow:Assets written to: saved_models\dropout_model\assets


In [30]:
OPT = Adam(lr_schedule)

BatchNorm_model = Sequential()
BatchNorm_model.add(normalisation_biotite_composition)
BatchNorm_model.add(Dense(128, activation="relu"))
BatchNorm_model.add(BatchNormalization())
BatchNorm_model.add(Dense(128, activation="relu"))
BatchNorm_model.add(BatchNormalization())
BatchNorm_model.add(Dense(128, activation="relu"))
BatchNorm_model.add(BatchNormalization())
BatchNorm_model.add(Dense(128, activation="relu"))
BatchNorm_model.add(BatchNormalization())
BatchNorm_model.add(Dense(128, activation="relu"))
BatchNorm_model.add(BatchNormalization())
BatchNorm_model.add(Dense(128, activation="relu"))
BatchNorm_model.add(BatchNormalization())
BatchNorm_model.add(Dense(2))

BatchNorm_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
BatchNorm_model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 6)                 13        
 on)                                                             
                                                                 
 dense_59 (Dense)            (None, 128)               896       
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                                  
                                                                 
 dense_60 (Dense)            (None, 128)               16512     
                                                                 
 batch_normalization_1 (Bat  (None, 128)               512       
 chNormalization)                                                
                                                     

In [31]:
history = BatchNorm_model.fit(biotite_composition_train, pt_train_norm,
                              batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                              validation_data=[biotite_composition_val, pt_val_norm],
                              callbacks=CALLBACKS_BatchNorm, verbose=False)
BatchNorm_model.save(Path("saved_models", "BatchNorm_model"))

INFO:tensorflow:Assets written to: saved_models\BatchNorm_model\assets


INFO:tensorflow:Assets written to: saved_models\BatchNorm_model\assets


In [32]:
OPT = Adam(lr_schedule)

LayerNorm_model = Sequential()
LayerNorm_model.add(normalisation_biotite_composition)
LayerNorm_model.add(Dense(128, activation="relu"))
LayerNorm_model.add(LayerNormalization())
LayerNorm_model.add(Dense(128, activation="relu"))
LayerNorm_model.add(LayerNormalization())
LayerNorm_model.add(Dense(128, activation="relu"))
LayerNorm_model.add(LayerNormalization())
LayerNorm_model.add(Dense(128, activation="relu"))
LayerNorm_model.add(LayerNormalization())
LayerNorm_model.add(Dense(128, activation="relu"))
LayerNorm_model.add(LayerNormalization())
LayerNorm_model.add(Dense(128, activation="relu"))
LayerNorm_model.add(LayerNormalization())
LayerNorm_model.add(Dense(2))

LayerNorm_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
LayerNorm_model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 6)                 13        
 on)                                                             
                                                                 
 dense_66 (Dense)            (None, 128)               896       
                                                                 
 layer_normalization (Layer  (None, 128)               256       
 Normalization)                                                  
                                                                 
 dense_67 (Dense)            (None, 128)               16512     
                                                                 
 layer_normalization_1 (Lay  (None, 128)               256       
 erNormalization)                                                
                                                     

In [33]:
history = LayerNorm_model.fit(biotite_composition_train, pt_train_norm,
                              batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                              validation_data=[biotite_composition_val, pt_val_norm],
                              callbacks=CALLBACKS_LayerNorm, verbose=False)
LayerNorm_model.save(Path("saved_models", "LayerNorm_model"))

INFO:tensorflow:Assets written to: saved_models\LayerNorm_model\assets


INFO:tensorflow:Assets written to: saved_models\LayerNorm_model\assets


In [34]:
OPT = Adam(lr_schedule)

l2_dropout_model = Sequential()
l2_dropout_model.add(normalisation_biotite_composition)
l2_dropout_model.add(Dense(128, activation="relu", kernel_regularizer="l2"))
l2_dropout_model.add(Dropout(0.2))
l2_dropout_model.add(Dense(128, activation="relu", kernel_regularizer="l2"))
l2_dropout_model.add(Dropout(0.2))
l2_dropout_model.add(Dense(128, activation="relu", kernel_regularizer="l2"))
l2_dropout_model.add(Dropout(0.2))
l2_dropout_model.add(Dense(128, activation="relu", kernel_regularizer="l2"))
l2_dropout_model.add(Dropout(0.2))
l2_dropout_model.add(Dense(128, activation="relu", kernel_regularizer="l2"))
l2_dropout_model.add(Dropout(0.2))
l2_dropout_model.add(Dense(128, activation="relu", kernel_regularizer="l2"))
l2_dropout_model.add(Dropout(0.2))
l2_dropout_model.add(Dense(2))

l2_dropout_model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)
l2_dropout_model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 6)                 13        
 on)                                                             
                                                                 
 dense_73 (Dense)            (None, 128)               896       
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_74 (Dense)            (None, 128)               16512     
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                                 
 dense_75 (Dense)            (None, 128)               16512     
                                                     

In [35]:
history = l2_dropout_model.fit(biotite_composition_train, pt_train_norm,
                              batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                              validation_data=[biotite_composition_val, pt_val_norm],
                              callbacks=CALLBACKS_L2_dropout, verbose=False)
l2_dropout_model.save(Path("saved_models", "L2_dropout_model"))

INFO:tensorflow:Assets written to: saved_models\L2_dropout_model\assets


INFO:tensorflow:Assets written to: saved_models\L2_dropout_model\assets
